In [1]:
import numpy as np
import pandas as pd
import re

## Clean Listings Data

In [2]:
# Loading in listing dats
listings_raw = pd.read_csv('../data/raw/listings.csv.xz', compression = 'xz')

# Drop most of the columns as they contain redundant or unnecessary information
col_to_drop = ['scrape_id', 'last_scraped', 'source', 'picture_url',
               'host_thumbnail_url', 'host_picture_url', 'latitude', 'longitude',
               'neighbourhood', 'minimum_minimum_nights', 'maximum_minimum_nights',
               'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm',
               'maximum_nights_avg_ntm', 'calendar_updated', 'availability_30', 'availability_60',
               'availability_90', 'availability_365', 'calendar_last_scraped', 'name',
               'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
               'last_review', 'license', 'instant_bookable', 'calculated_host_listings_count',
               'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
               'calculated_host_listings_count_shared_rooms', 'reviews_per_month',
               'host_has_profile_pic', 'host_identity_verified', 'host_listings_count',
               'neighborhood_overview', 'host_about', 'host_response_time', 'host_response_rate',
               'host_acceptance_rate', 'review_scores_accuracy', 'review_scores_cleanliness',
               'review_scores_checkin', 'review_scores_communication', 'review_scores_location',
               'review_scores_value', 'host_location', 'host_neighbourhood']
listings_raw = listings_raw.drop(col_to_drop, axis = 1)

/tmp/ipykernel_71182/1394727334.py:2: DtypeWarning: Columns (0,22,41) have mixed types. Specify dtype option on import or set low_memory=False.
  listings_raw = pd.read_csv('../data/raw/listings.csv.xz', compression = 'xz')


In [3]:
# Check for null values
listings_raw.isnull().sum()

id                                  0
listing_url                         1
description                      1044
host_id                             0
host_url                            0
host_name                           5
host_since                          5
host_is_superhost                 486
host_total_listings_count           6
host_verifications                  7
neighbourhood_cleansed              1
neighbourhood_group_cleansed        1
property_type                       1
room_type                           1
accommodates                        1
bathrooms                       14810
bathrooms_text                     32
bedrooms                         5984
beds                            14952
amenities                           1
price                           14807
minimum_nights                      1
maximum_nights                      1
has_availability                 5368
number_of_reviews                   1
review_scores_rating            11561
dtype: int64

In [4]:
# Assume hosts are not superhosts by default
listings_raw.host_is_superhost.unique()

array(['f', 't', nan, 'Private room in rental unit'], dtype=object)

In [5]:
# Assume listing does not have availability by default
listings_raw.has_availability.unique()

array(['t', nan, 'f'], dtype=object)

In [6]:
# All listings with null review scores have zero reviews
listings_raw[listings_raw.review_scores_rating.isnull()]['number_of_reviews'].value_counts()

number_of_reviews
0.0    11559
1.0        1
Name: count, dtype: int64

In [7]:
# Basic filling of missing values
listings_raw.description = listings_raw.description.fillna('No description')
listings_raw.host_is_superhost = listings_raw.host_is_superhost.fillna('f')
listings_raw.has_availability = listings_raw.has_availability.fillna('f')
listings_raw.review_scores_rating = listings_raw.review_scores_rating.fillna(0.0)

In [8]:
listings_raw.bathrooms_text.unique()

array(['1 bath', '1 shared bath', '1 private bath', '2 baths',
       '1.5 baths', '3 baths', '2 shared baths', '2.5 baths', nan,
       '1.5 shared baths', '5 baths', '0 shared baths',
       'Shared half-bath', '4.5 baths', '2.5 shared baths', 'Half-bath',
       'Private half-bath', '0 baths', '3.5 baths', '3.5 shared baths',
       '15.5 baths', '3 shared baths', '10.5 baths', '4 baths',
       '4 shared baths', '4.5 shared baths', '5.5 baths', '6 baths',
       '7 baths', '11.5 baths', '6 shared baths', '7.5 baths',
       '7.5 shared baths', '5 shared baths', '6.5 baths'], dtype=object)

In [9]:
# Extract the number of bathrooms from bathroom_text, where applicable
listings_raw.bathrooms_text = listings_raw.bathrooms_text.fillna('0 baths')
listings_raw.loc[listings_raw.bathrooms.isnull(), 'bathrooms'] = listings_raw.bathrooms_text.apply(
                                            lambda x: re.search('^(\d+)', x)[1]
                                            if re.search('^(\d+)', x) is not None
                                            else np.nan)

In [10]:
# Assume there is one bed per bedroom, on average
listings_raw.bedrooms = listings_raw.bedrooms.fillna(0)
listings_raw.loc[listings_raw.beds.isnull(), 'beds'] = listings_raw.bedrooms

In [11]:
# There is no discernable way to recover price
listings_raw[listings_raw.price.isnull()][:5]

,id,listing_url,description,host_id,host_url,host_name,host_since,host_is_superhost,host_total_listings_count,host_verifications,...,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,has_availability,number_of_reviews,review_scores_rating
4,7064,https://www.airbnb.com/rooms/7064,"Large, private loft-like room in a spacious 2-...",17297.0,https://www.airbnb.com/users/show/17297,Joelle,2009-05-15,f,2.0,"['email', 'phone', 'work_email']",...,1 shared bath,1.0,1.0,"[""Heating"", ""Washer"", ""Dishes and silverware"",...",NaN,30,45.0,t,13.0,4.91
10,11943,https://www.airbnb.com/rooms/11943,No description,45445.0,https://www.airbnb.com/users/show/45445,Harriet,2009-10-12,f,3.0,['phone'],...,1 bath,1.0,1.0,"[""Heating"", ""Washer"", ""Air conditioning"", ""Bre...",NaN,30,730.0,f,0.0,0.00
17,15396,https://www.airbnb.com/rooms/15396,"Modern, spacious, bright and well designed apa...",60278.0,https://www.airbnb.com/users/show/60278,Petra,2009-12-05,t,3.0,"['email', 'phone']",...,2 baths,2.0,2.0,"[""Fire extinguisher"", ""Smoke alarm"", ""Sound sy...",NaN,180,730.0,t,5.0,5.00
24,18961,https://www.airbnb.com/rooms/18961,"This is a great fully furnished studio, lots o...",70857.0,https://www.airbnb.com/users/show/70857,Fabian,2010-01-15,f,2.0,['phone'],...,1 bath,1.0,1.0,"[""TV with standard cable"", ""Heating"", ""Essenti...",NaN,30,180.0,f,1.0,5.00
26,19282,https://www.airbnb.com/rooms/19282,"A true GEM in Prospect Lefferts Gardens, welco...",73469.0,https://www.airbnb.com/users/show/73469,Djassi,2010-01-25,f,1.0,"['email', 'phone']",...,2 baths,0.0,0.0,"[""Fire extinguisher"", ""Smoke alarm"", ""Stove"", ...",NaN,30,60.0,t,16.0,4.69


In [12]:
# Drop rows with missing values from these columns
cols_drop_null = ['host_name', 'host_since', 'listing_url', 'host_verifications', 'price']
for c in cols_drop_null:
    listings_raw = listings_raw[~listings_raw[c].isnull()]

In [13]:
# Drop bathroom_text column, since it is redundant with number of baths
listings_raw = listings_raw.drop(['bathrooms_text'], axis = 1)

In [14]:
listings_raw.isnull().sum()

id                              0
listing_url                     0
description                     0
host_id                         0
host_url                        0
host_name                       0
host_since                      0
host_is_superhost               0
host_total_listings_count       0
host_verifications              0
neighbourhood_cleansed          0
neighbourhood_group_cleansed    0
property_type                   0
room_type                       0
accommodates                    0
bathrooms                       0
bedrooms                        0
beds                            0
amenities                       0
price                           0
minimum_nights                  0
maximum_nights                  0
has_availability                0
number_of_reviews               0
review_scores_rating            0
dtype: int64

In [15]:
listings_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22741 entries, 0 to 37548
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            22741 non-null  object 
 1   listing_url                   22741 non-null  object 
 2   description                   22741 non-null  object 
 3   host_id                       22741 non-null  float64
 4   host_url                      22741 non-null  object 
 5   host_name                     22741 non-null  object 
 6   host_since                    22741 non-null  object 
 7   host_is_superhost             22741 non-null  object 
 8   host_total_listings_count     22741 non-null  float64
 9   host_verifications            22741 non-null  object 
 10  neighbourhood_cleansed        22741 non-null  object 
 11  neighbourhood_group_cleansed  22741 non-null  object 
 12  property_type                 22741 non-null  object 
 13  room_t

In [16]:
# Fix data types
int_col = ['id', 'host_id', 'host_total_listings_count',
           'accommodates', 'bedrooms', 'beds', 
           'minimum_nights', 'maximum_nights', 'number_of_reviews']
for c in int_col:
    listings_raw[c] = listings_raw[c].astype(int)

listings_raw.price = listings_raw.price.apply(lambda x: float(''.join(x[1:].split(','))))

## Clean Reviewers/Users Data

In [17]:
# Load reviews data and subselect desired columns
reviews_raw = pd.read_csv('../data/raw/reviews.csv.xz', compression = 'xz')
reviews_raw = reviews_raw[['listing_id', 'date', 'reviewer_id', 'reviewer_name']]

/tmp/ipykernel_71182/1876831970.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  reviews_raw = pd.read_csv('../data/raw/reviews.csv.xz', compression = 'xz')


In [18]:
# Only include data with listings in the cleaned listing data so that appropriate connections can be made
reviews_raw = reviews_raw.loc[reviews_raw.listing_id.isin(listings_raw.id.unique())]

In [19]:
# Investigate null values
reviews_raw.isnull().sum()

listing_id       0
date             0
reviewer_id      0
reviewer_name    1
dtype: int64

In [20]:
# Remove null values
reviews_raw = reviews_raw[~reviews_raw.isnull()]

In [21]:
reviews_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 471540 entries, 393223 to 965180
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   listing_id     471540 non-null  object 
 1   date           471540 non-null  object 
 2   reviewer_id    471540 non-null  float64
 3   reviewer_name  471539 non-null  object 
dtypes: float64(1), object(3)
memory usage: 18.0+ MB


In [22]:
# Fix data types
int_col = ['listing_id', 'reviewer_id']
for c in int_col:
    reviews_raw[c] = reviews_raw[c].astype(int)

In [23]:
# Write to csv
reviews_raw.to_csv('../data/cleaned/reviewers.csv.xz', index = False)

## Splitting Listings Into Listings and Hosts "Tables"

In [24]:
# Select columns relevant to listings, including host id for relationships
listings_col = ['id', 'listing_url', 'description', 'host_id',
                'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'property_type', 'room_type',
                'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'price', 
                'minimum_nights', 'maximum_nights', 'has_availability', 'number_of_reviews',
                'review_scores_rating']
listings_cleaned = listings_raw.copy()[listings_col]

# Write to csv
listings_cleaned.to_csv('../data/cleaned/listings.csv.xz', index = False)

In [25]:
# Select columns relevant to hosts
hosts_col = ['host_id', 'host_url', 'host_name', 'host_since', 'host_is_superhost',
             'host_total_listings_count', 'host_verifications']
hosts_cleaned = listings_raw.copy()[hosts_col].drop_duplicates()

# Write to csv
hosts_cleaned.to_csv('../data/cleaned/hosts.csv.xz', index = False)